In [213]:
### Grading script code 
### You don't need to read this, proceed to the next cell
import sys
import functools
ipython = get_ipython()

def set_traceback(val):
    method_name = "showtraceback"
    setattr(
        ipython,
        method_name,
        functools.partial(
            getattr(ipython, method_name),
            exception_only=(not val)
        )
    )

class AnswerError(Exception):
  def __init__(self, message):
    pass

def exec_test(f, question):
    try:
        f()
        print(question + " Pass")
    except:
        set_traceback(False) # do not remove
        raise AnswerError(question + " Fail")

# Week 6 Problem Set

## Homeworks

**HW1.** Extend the class `Fraction` to implement the other operators: `- * < <= > >=`.

In [214]:
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)

class Fraction:
    def __init__(self, num, den):
        self.num = num
        self.den = den
    
    @property
    def num(self):
        return self._num
    
    @num.setter
    def num(self, val):
        # if the val is not an integer, do nothing 
        if type(val) != int:
            return
        self._num = val
    
    @property
    def den(self):
        return self._den
    
    @den.setter
    def den(self, val):
        if type(val) != int or val == 0:
            self._den = 1
            return
        self._den = val
    
    def __str__(self):
        return f'{self.num}/{self.den}'
    
    def simplify(self):
        # according to requirements
        divisor = gcd(self.num, self.den)
        return Fraction(self.num//divisor, self.den//divisor)
    
    def __add__(self, other):
        # it's math
        return Fraction( (self.num * other.den + other.num * self.den) , (self.den * other.den) ).simplify()
        
    def __eq__(self, other):
        f1 = self.simplify()
        f2 = other.simplify()
        return f1.num == f2.num and f1.den == f2.den
    
    def __sub__(self, other):
        return Fraction( (self.num * other.den - other.num * self.den) , (self.den * other.den) ).simplify()
    
    def __mul__(self, other):
        return Fraction( (self.num * other.num) , (self.den * other.den) ).simplify()
    
    def __lt__(self, other):
        return self.num * other.den < other.num * self.den
    
    def __le__(self, other):
        return self.num * other.den <= other.num * self.den
    
    def __gt__(self, other):
        return self.num * other.den > other.num * self.den
    
    def __ge__(self, other):
        return self.num * other.den >= other.num * self.den
    


In [215]:
f1 = Fraction(3, 4)
f2 = Fraction(1, 2)
f3 = f1 - f2
assert f3 == Fraction(1, 4)
f4 = f1 * f2
assert f4 == Fraction(3, 8)
assert f2 < f1
assert f2 <= f2
assert f1 > f3
assert f3 >= f3

In [216]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [217]:
class MixedFraction(Fraction):
    def __init__(self, top, bot, whole=0):
        num = top + whole * bot if whole else top
        super().__init__(num, bot)

    def get_three_numbers(self):
        # math
        whole = self.num // self.den
        top = self.num % self.den
        bot = self.den

        return (top, bot, whole)

    def __str__(self):
        if self.num > self.den:
            tup = self.get_three_numbers()
            return f"{tup[-1]} {tup[0]}/{tup[1]}"
            
        return f"{self.num}/{self.den}"

In [218]:
mf1 = MixedFraction(5, 3)
assert mf1.num == 5 and mf1.den == 3
assert mf1.get_three_numbers() == (2, 3, 1)
mf2 = MixedFraction(2, 3, 1)
assert mf2.num == 5 and mf2.den == 3

result = mf1 + mf2
assert result.num == 10 and result.den == 3

result = mf1 * mf2
assert result.num == 25 and result.den == 9

mf3 = MixedFraction(1, 2, 1)
result = mf1 - mf3
assert result.num == 1 and result.den == 6

assert str(mf1) == "1 2/3"

In [219]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Write a class called `EvaluateFraction` that evaluates postfix notation implemented using Dequeue data structures only. Postfix notation is a way of writing expressions without using parenthesis. For example, the expression `(1+2)*3` would be written as `1 2 + 3 *`. The class `EvaluateFraction` has the following method:
- `input(inp)`: which pushes the input input one at a time. For example, to create a postfix notation `1 2 + 3 *`, we can call this method repetitively, e.g. `e.input('1'); e.input('2'); e.input('+'); e.input('3'); e.input('*')`. Notice that the input is of String data type. 
- `evaluate()`: which returns the output of the expression.
- `get_fraction(inp)`: which takes in an input string and returns a `Fraction` object. 

Postfix notation is evaluated using a Stack. Since `Dequeue` can be used for both Stack and Queue, we will implement using `Dequeue`. The input streams from `input()` are stored in a Queue, which we will again implement using Dequeue. If the output of the Queue is a number, the item is pushed into the stack. If it is an operator, we will apply the operator to the two top most item n the stacks and push the result back into the stack. 

In [220]:
class Stack:
    def __init__(self):
        self.ls = []
    def push(self, item):
        self.ls.append(item)
    def pop(self):
        return self.ls.pop()
    def peek(self):
        return self.ls[-1]

    @property
    def is_empty(self):
        return self.ls == []


In [221]:
class Queue:
    def __init__(self):
        self.left_stack = Stack()
        self.right_stack = Stack()

    @property
    def is_empty(self):
        return self.left_stack.is_empty and self.right_stack.is_empty

    @property
    def size(self):
        return len(self.left_stack.ls) + len(self.right_stack.ls)

    def enqueue(self, item):
        self.left_stack.push(item)

    def dequeue(self):
        if self.right_stack.is_empty:
            while not self.left_stack.is_empty:
                self.right_stack.push(self.left_stack.pop())
        return self.right_stack.pop()
    
    def peek(self):
        if self.right_stack.is_empty:
            while not self.left_stack.is_empty:
                self.right_stack.push(self.left_stack.pop())
        return self.right_stack[-1]

In [222]:
class Deque(Queue):
  
    def add_front(self, item):
        self.right_stack.push(item)
      
    def remove_front(self):
        return self.dequeue()
    
    def add_rear(self, item):
        self.enqueue(item)

    def left_to_right(self):
        if self.left_stack.is_empty:
            return
        temp = Stack()

        while not self.right_stack.is_empty:
            temp.push(self.right_stack.pop())
        while not self.left_stack.is_empty:
            self.right_stack.push(self.left_stack.pop())
        while not temp.is_empty:
            self.right_stack.push(temp.pop())
        
    def remove_rear(self):  
        self.left_to_right()
        return self.right_stack.ls.pop(0)

    def peek_front(self):
        self.left_to_right()
        return self.right_stack.ls[-1]
    
    def peek_rear(self):
        self.left_to_right()
        return self.right_stack.ls[0]

In [223]:

class EvaluateFraction:

    operands = "0123456789"
    operators = "+-*/"
    operations = {"+": lambda x,y: y + x,
                  "-": lambda x,y: y - x,
                  "*": lambda x,y: y * x,
                  "/": lambda x,y: y / x}
    
    def __init__(self):
        self.expression = Deque()
        self.stack = Deque()
    
    def input(self, item):
        # if item is not an operator, convert it to fraction
        if item not in self.operations:
            item = self.get_fraction(item)
        # add everything into queue
        self.expression.add_rear(item)

    def evaluate(self):
        while not self.expression.is_empty:
            # dequeue from self.expression
            item = self.expression.remove_front()
            # if item is a string, meaning an operator, 
            if type(item) == str:
                # process the operator, meaning, remove 2 from the top of stack and do process_operator, result is saved into item
                item = self.process_operator(self.stack.remove_rear(), self.stack.remove_rear(), item)
            # item is added to the stack, it will always be a Fraction added
            self.stack.add_rear(item)
        # at the end, there should only be one Fraction left in stack, so just remove it 
        return self.stack.remove_front()

    def get_fraction(self, inp):
        # this splits the two numbers in e.g "1/2" to [1, 2]
        num, den = inp.split("/")
        return Fraction(int(num), int(den))

    def process_operator(self, op1, op2, op):
        return self.operations[op](op1, op2)


In [224]:
pe = EvaluateFraction()
pe.input("1/2")
pe.input("2/3")
pe.input("+")
assert pe.evaluate()==Fraction(7, 6)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
assert pe.evaluate()==Fraction(1, 1)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
pe.input("3/4")
pe.input("*")
assert pe.evaluate()==Fraction(3, 4)

In [225]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** Modify HW2 so that it can work with MixedFraction. Write a class called `EvaluateMixedFraction` as a subclass of `EvaluateFraction`. You need to override the following methods:
- `get_fraction(inp)`: This function should be able to handle string input for MixedFraction such as `1 1/2` or `3/2`. It should return a `MixedFraction` object.
- `evaluate()`: This function should return `MixedFraction` object rather than `Fraction` object. 

In [226]:
class EvaluateMixedFraction(EvaluateFraction):
    def get_fraction(self, inp):
        numbers = inp.replace('/', " ").split()
        if len(numbers) == 3:
            return MixedFraction(int(numbers[1]), int(numbers[2]), int(numbers[0]))
        return MixedFraction(int(numbers[0]), int(numbers[1]))
    
    def evaluate(self):
        while not self.expression.is_empty:
            # dequeue from self.expression
            item = self.expression.remove_front()
            # if item is a string, meaning an operator, 
            if type(item) == str:
                # process the operator, meaning, remove 2 from the top of stack and do process_operator, result is saved into item
                item = self.process_operator(self.stack.remove_rear(), self.stack.remove_rear(), item)
            # item is added to the stack, it will always be a Fraction added
            self.stack.add_rear(item)
        # at the end, there should only be one Fraction left in stack, so just remove it 
        return self.stack.remove_front()

In [227]:
pe = EvaluateMixedFraction()
pe.input("3/2")
pe.input("1 2/3")
pe.input("+")
assert pe.evaluate() == MixedFraction(1, 6, 3)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1 1/8")
pe.input("-")
assert pe.evaluate() == MixedFraction(1, 24)

pe.input("1 1/2")
pe.input("2 2/3")
pe.input("+")
pe.input("1 1/6")
pe.input("-")
pe.input("5/4")
pe.input("*")
assert pe.evaluate() == MixedFraction( 3, 4, 3)

In [228]:
###
### YOUR CODE HERE
###


**HW4.** *Linked List:* We are going to implement Linked List Abstract Data Type. To do so, we will implement two classes: `Node` and `MyLinkedList`. In this part, we will implement the class Node.

The class `Node` has the following attribute and computed property:
- `element`: which stores the value of the item in that node.
- `next`: which stores the reference to the next `Node` in the list. The setter method should check if the value assigned is of type `Node`.





In [229]:
class Node:
    def __init__(self, e):
        self.element = e
        self.__next = None
               
    @property
    def next(self):
        return self.__next
    
    @next.setter
    def next(self, value):
        # check if value is an instance of Node object
        # you can use isinstance() function
        if isinstance(value, Node):
            self.__next = value


**HW5.** This is a continuation to implement a Linked List. The class `MyLinkedList` has two different properties:
- `head`: which points to the `Node` of the first element.
- `tail`: which points to the `Node` of the last element.

It should also have the following methods:
- `__init__(items)`: which create the link list object based using the arguments.
- `_get(index)`: which returns the item at the given `index`.
- `_add_first(item)`: which adds the `item` as the first element.
- `_add_last(item)`: which adds the `item` as the last element.
- `_add_at(index, item)`: which adds the `item` at the position `index`.
- `_remove_first(item)`: which removes the `item` as the first element.
- `_remove_last(item)`: which removes the `item` as the last element.
- `_remove_at(index, item)`: which removes the `item` at the position `index`.


In [230]:
import collections.abc as c

class MyAbstractList(c.Iterator):
    
    def __init__(self, list_items):
        # iterate over every element and call self.append(item)
        self.size = 0
        self._idx = 0
        for i in list_items:
            self.append(i)
    
    
    @property
    def is_empty(self):
        return self.size == 0
    
    def append(self, item):
        self._add_at(self.size, item)
        
    def remove(self, item):
        idx = self._index_of(item)
        # _index_of returns -1 if the item is not in the data, so we need to account for -1 which we are not supposed to remove
        if idx >= 0:
            self._remove_at(idx)
        
    def __getitem__(self, index):
        return self._get(index)
    
    def __setitem__(self, index, value):
        self._set_at(index, value)
        
    def __delitem__(self, index):
        self._remove_at(index)
    
    def __len__(self):
        return self.size
        
    def __iter__(self):
        self._idx = 0
        return self
        
    def __next__(self):
        if self._idx < self.size:
            n_item = self._get(self._idx)
            self._idx += 1
            return n_item
        else:
            raise StopIteration
    
    # the following methods should be implemented in the child class
    def _get(self, index):
        pass

    def _set_at(self, index, item):
        pass

    def _add_at(self, index, item):
        pass

    def _remove_at(self, index):
        pass

    def _index_of(self, item):
        pass

In [234]:
class MyLinkedList(MyAbstractList):
    def __init__(self, items):
        self.head = None
        self.tail = None
        super().__init__(items)
        
    def _get(self, index):
        # do the following:
        # 1. traverse to the node at index
        # 2. return the element of that node
        curr = self._goto(index)
        return curr.element
    
    def _goto(self, index):
        # FROM INDEX 0
        curr = self.head
        # TRAVERSE TO INDEX
        for i in range(index):
            curr = curr.next
        return curr
    
    def _add_first(self, element):
        # do the following:
        # 1. create a new Node object using element
        # 2. set the current head reference as the next reference of the new node
        # 3. increase size by 1
        # 4. if this is the last element (no tail) -> set the current node as the tail
        new = Node(element)
        new.next = self.head
        if not self.head:
            self.tail = new
        self.head = new
        self.size += 1
        
    def _add_last(self, element):
        # do the following:
        # 1. create a new Node object using element
        # 2. if there is no element as tail -> set the new node as both
        #    the tail and the head
        # 3. otherwise, -> 
        #    - set the new node as the next reference of the tail
        #    - set new node as tail
        '''
        TAKE NOTE!!! 
        this is in the ogiginal instructions,
        set the next reference of the current node as the tail's next reference (what the are they smoking???? this is wrong)
        '''
        # 4. increase size by 1
        new = Node(element)
        if not self.tail:
            self.head = new
        else:
            self.tail.next = new
        self.tail = new
        self.size += 1
        
    def _add_at(self, index, element):
        if index == 0:
            self._add_first(element)

        elif index >= self.size:
            # if insert at last position or more, call add_last() method
            self._add_last(element)
            
        else:
            # if insert in between, do the following:
            # 1. start from the head, traverse the linked list to get
            #    the reference at position index-1 using its next reference
            # 2. create a new Node
            # 3. set the next of the current node as the next of the new Node
            # 4. set the new node as the next of the current node
            # 5. increase the size by 1
            curr = self._goto(index-1)
            new = Node(element)
            new.next = curr.next
            curr.next = new
            self.size += 1

    def _set_at(self, index, element):
        if 0 <= index < self.size:
            current = self._goto(index)
            current.element = element
            
    def _remove_first(self):
        if self.size == 0:
            # if list is empty, return None
            return None
        else:
            # otherwise, do the following:
            # 1. store the head at a temporary variable
            # 2. set the next reference of the current head to be the head
            # 3. reduce size by 1
            # 4. if the new head is now None, it means empty list
            #    -> set the tail to be None also
            # 5. return element of the removed node
            temp = self.head
            self.head = self.head.next
            self.size -= 1
            if not self.head:
                self.tail = None
            return temp.element
        
        
    def _remove_last(self):
        if self.size == 0:
            # if the list is empty, return None
            return None
        elif self.size == 1:
            # if there is only one element, just remove that one node 
            # using some other method
            return self._remove_first()

        else:
            # otherwise, do the following:
            # 1. traverse to the second last node
            # 2. store the tail of the list to a temporary variable
            # 3. set the current node as the tail
            # 4. set the next ref of the tail to be None
            # 5. reduce the size by 1
            # 6. return the element of the removed node in the temp var
            curr = self._goto(self.size-2)
            temp = self.tail
            self.tail = curr
            self.tail.next = None
            self.size -= 1
            return temp.element

    def _remove_at(self, index):
        if index < 0 or index >= self.size:
            return None
        elif index == 0:
            return self._remove_first()
        elif index == self.size - 1:
            return self._remove_last()
        else:
            # do the following:
            # 1. traverse to the node at index - 1
            # 2. get the node at index using next reference
            # 3. set the next node of the node at index - 1
            # 4. decrease the size by 1
            # 5. return the element that is removed
            curr = self._goto(index-1)
            temp = curr.next
            curr.next = temp.next
            self.size -= 1
            return temp.element

In [235]:
asean = MyLinkedList(['Singapore', 'Malaysia'])
assert asean.head.element == 'Singapore'
assert asean.tail.element == 'Malaysia'
asean.append('Indonesia')
assert asean.tail.element == 'Indonesia'
asean._add_at(0, 'Brunei')
assert asean.head.element == 'Brunei'
assert asean.size == 4
assert len(asean) == 4
asean[0] = 'Cambodia'
assert asean[0] == 'Cambodia' and asean[1] == 'Singapore'
asean[2] = 'Myanmar'
assert(len(asean)) == 4 
assert [x for x in asean] == ['Cambodia', 'Singapore', 'Myanmar', 'Indonesia']

del asean[0]
assert [x for x in asean] == ['Singapore', 'Myanmar', 'Indonesia']

asean._add_at(2, 'Brunei')
assert [x for x in asean] == ['Singapore', 'Myanmar', 'Brunei', 'Indonesia']
del asean[3]
assert [x for x in asean] == ['Singapore', 'Myanmar', 'Brunei']
del asean[1]
assert [x for x in asean] == ['Singapore', 'Brunei']
del asean[1]
assert [x for x in asean] == ['Singapore']
del asean[0]
assert [x for x in asean] == []

In [233]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
